                                                 Домашнє завдання N 7

Так як ми працюємо з рекомендованними системами, по завданню нам радять використовувати бібліотеку Surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Тому давайте імпортуємо цю бібліотеку і також завантажимо потрібний датасет movielens на 100 к даних. Думаю, нам буде достатньо цих даних для виконання домашнього завдання:

In [1]:
from surprise import Dataset, Reader, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [2]:
dataset_path = './ml-100k/u.data'
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(dataset_path, reader=reader)

data

Давайте спробуємо вивести перші 5 рядків, щоб побачити, які дані містить наш датасет. Для цього використаємо трохи інший ніж зазвичай метод виводу:

In [3]:
N = 5
data_raw = data.raw_ratings

for i in range(N):
    print(data_raw[i])



('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


Окей, супер!

Тепер давайте побудуємо модел за допомогою 3 алгоритмів розграхунків: SVD, SVD++, NMF.

Почнемо з SVD:

In [4]:
algo = SVD()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9389  0.9355  0.9408  0.9407  0.9363  0.9384  0.0022  
MAE (testset)     0.7407  0.7387  0.7416  0.7411  0.7385  0.7401  0.0013  
Fit time          0.38    0.39    0.40    0.42    0.40    0.40    0.01    
Test time         0.04    0.04    0.06    0.06    0.04    0.05    0.01    


{'test_rmse': array([0.93885224, 0.93545571, 0.94075718, 0.94068884, 0.93626869]),
 'test_mae': array([0.74072931, 0.73871931, 0.74161265, 0.74109386, 0.73853116]),
 'fit_time': (0.3798980712890625,
  0.39232897758483887,
  0.3993709087371826,
  0.41629481315612793,
  0.39879703521728516),
 'test_time': (0.04043889045715332,
  0.039871931076049805,
  0.05989193916320801,
  0.06384015083312988,
  0.03844618797302246)}

Тепер те саме зробимо з SVD++:

In [5]:
algo_svdpp = SVDpp()

cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9242  0.9239  0.9156  0.9177  0.9198  0.9203  0.0034  
MAE (testset)     0.7266  0.7259  0.7172  0.7183  0.7200  0.7216  0.0039  
Fit time          6.22    6.11    6.13    6.11    6.16    6.15    0.04    
Test time         1.29    1.31    1.36    1.26    1.31    1.31    0.03    


{'test_rmse': array([0.92424952, 0.92388592, 0.91563535, 0.91770312, 0.91980131]),
 'test_mae': array([0.72657472, 0.72594443, 0.71715336, 0.71832787, 0.71999657]),
 'fit_time': (6.221038103103638,
  6.107075929641724,
  6.132678031921387,
  6.107383966445923,
  6.162589073181152),
 'test_time': (1.2880079746246338,
  1.305907964706421,
  1.3591089248657227,
  1.2617161273956299,
  1.3107430934906006)}

І також використовуючи NMF модель:

In [6]:
algo_nmf = NMF()

cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9592  0.9630  0.9742  0.9598  0.9510  0.9615  0.0075  
MAE (testset)     0.7535  0.7567  0.7663  0.7556  0.7472  0.7558  0.0062  
Fit time          0.49    0.53    0.49    0.47    0.50    0.50    0.02    
Test time         0.06    0.06    0.03    0.06    0.06    0.05    0.01    


{'test_rmse': array([0.95924562, 0.9630369 , 0.97415415, 0.95984841, 0.95098141]),
 'test_mae': array([0.75348726, 0.75667046, 0.76629095, 0.7555553 , 0.74719094]),
 'fit_time': (0.490278959274292,
  0.5348570346832275,
  0.48781585693359375,
  0.46695566177368164,
  0.49598097801208496),
 'test_time': (0.060131072998046875,
  0.06031012535095215,
  0.03175926208496094,
  0.05784320831298828,
  0.06034278869628906)}

Ми отримали результати оцінок RMSE та MAE всіх 3 алгоритмів. Давайте проаналізуємо отримані дані та зробимо висновки:

Алгоритм SVD: Має середнє значення RMSE приблизно 0.9345 і MAE приблизно 0.7373. Цей алгоритм показав непогані результати з помірним часом навчання та тестування.

Алгоритм SVD++: Показав кращі результати за RMSE (середнє 0.9194) та MAE (середнє 0.7213) порівняно з SVD, що свідчить про вищу точність прогнозування. Однак, час навчання та тестування значно вищий, що є важливим аспектом для великих наборів даних або обмежених обчислювальних ресурсів.

Алгоритм NMF: Має найвищі значення помилок RMSE (середнє 0.9648) та MAE (середнє 0.7572) серед трьох розглянутих алгоритмів, що вказує на нижчу точність прогнозування порівняно з SVD та SVD++. Час навчання та тестування подібний до SVD.

Як висновок, можемо зазначити, що найбільш вдалим тут здається вибір алгоритму SVD++, так як в нас датасет не такий великий. Якби ми взяли датасет не на 100к а на 1М даних, то досить велика ймовірність, що краще б було тоді застосовувати алгоритм SVD.

Ми знайшли оптимальний алгоритм на основі пошуку параметрів RMSE,MAE через cross_validation.

Тепер же давайте застосуємо метод GridSearchCV для пошуку оптимальних гіперпараметрів. Для цього використаємо розділ документації бібліотеки surprise: Tune algorithm parameters with GridSearchCV.

Почнемо з алгоритму SVD++, так як він показався найбільш оптимальним, хоча достатньо ресурсно-залежний:

In [12]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse = SVDpp(**gs.best_params['rmse'])
algo_mae = SVDpp(**gs.best_params['mae'])

RMSE: 0.9643501390059793
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE: 0.7728093432790312
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
